In [ ]:
import autosklearn.classification
import sklearn.model_selection
import sklearn.datasets
import sklearn.metrics

X, y = sklearn.datasets.load_digits(return_X_y=True)
X_train, X_test, y_train, y_test = \
        sklearn.model_selection.train_test_split(X, y, random_state=1)
automl = autosklearn.classification.AutoSklearnClassifier(time_left_for_this_task=31)
automl.fit(X_train, y_train)
y_hat = automl.predict(X_test)

print(sklearn.metrics.accuracy_score(y_test, y_hat))

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import _pickle as pickle
# import tensorflow as tf
def save(file,name, folder = ""):
    if folder != "":
        outfile = open('./'+folder+'/'+name+'.pickle', 'wb')
    else:
        outfile = open(name+'.pickle', 'wb')
    pickle.dump(file, outfile, protocol=4)
    outfile.close
    
def load(name, folder = ""):
    if folder != "":
        outfile = open('./'+folder+'/'+name+'.pickle', 'rb')
    else:
        outfile = open(name+'.pickle', 'rb')
    file = pickle.load(outfile)
    outfile.close
    return file

pd.set_option('display.max_columns', 150)
pd.set_option('display.max_rows', 150)

In [ ]:
import autosklearn.classification

In [ ]:
1+1

In [ ]:
## Time series version (date < 400 et > 400)
2000000 train values, 323000 test values
Max possible 29847

xgboost bce loss 500 trees  utility : 2184 / sw : 2389
catboost bce loss 2000 trees utility : 2435 / sw : 1845
lgb bce loss 29 trees utility : 2277 / sw : 2645

In [ ]:
df = load('train')

In [ ]:
df_t = pd.read_csv('example_test.csv')

In [ ]:
df_f = pd.read_csv('features.csv')

In [ ]:
df_f.head()

In [ ]:
dico_feats_tag = {}

for i, line in df_f.iterrows():
    f = line['feature']
    
    t = " ".join(list(df_f.columns[1:][line[df_f.columns[1:]].values]))
    
    dico_feats_tag[f] = t

In [ ]:
df.head()

In [ ]:
df = df.fillna(-100)

In [2]:
def utility_metric(date,weights, resp, action):
    import numpy as np
    p = []
    for i in np.unique(date):
        wi = weights[date == i]
        ri = resp[date == i]
        ai = action[date == i]
        pi = np.sum(wi * ri * ai)
        p.append(pi)
    p = np.array(p)
    
    nt = np.unique(date).shape[0]
#     print(nt)
    sp = np.sum(p)
    normp = np.sqrt(np.sum(np.square(p)))
    t = (sp / normp) * np.sqrt(250/nt)
    u = min(max(t,0), 6) * sp
    return u
    

In [ ]:
df.head()

In [ ]:
df = df[df['weight'] != 0]

In [ ]:
df.shape

In [ ]:
train = df[df['date']<440]
test = df[df['date']>=440]

train = train.sample(n = train.shape[0])
test = test.sample(n = test.shape[0])

X_train = train[['weight'] + ['feature_'+str(i) for i in range(130)]]
X_test = test[['weight'] + ['feature_'+str(i) for i in range(130)]]

date_train = train['date'].values
date_test = test['date'].values

weights_train = train['weight'].values
weights_test = test['weight'].values

y_train = train['resp'].values
y_test = test['resp'].values

In [ ]:
action = (y_test> 0)*1
import random
utility_metric(date_test,weights_test,y_test, action)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
save((X_train, X_test, y_train, y_test, date_train, date_test, weights_train, weights_test), 'splitted_dataset')

## Data

In [ ]:
(X_train, X_test, y_train, y_test, date_train, date_test, weights_train, weights_test) = load('splitted_dataset')
SAMPLE_WEIGHTS = abs((y_train * weights_train)) + 1
sample_weights = abs((np.concatenate([y_train, y_test]) * np.concatenate([weights_train, weights_test]))) + 1

## Tree Based

In [ ]:
X_train = X_train
# X_train['label'] = (y_train > 0)*1
y_train1 = (y_train > 0)*1

# X_val = X_train[date_train >= 420]
# X = X_train[date_train <420]

X_test = X_test#[:100]
y_test1 = (y_test > 0)*1

# X_test['label'] = y_test1

# X_test = X_test


In [ ]:
import gc
del X_train
gc.collect()

In [ ]:
X_val.shape

In [ ]:
X.shape

In [ ]:
X_test.shape

In [ ]:
## Params on https://auto.gluon.ai/stable/api/autogluon.task.html#autogluon.task.TabularPrediction

import autogluon as ag
from autogluon import TabularPrediction as task

# dir1 = 'agModels-predictClass'
# predictor = task.fit(train_data=X, label='label', output_directory=dir1)

presets = ['best_quality',
           'best_quality_with_high_quality_refit', #x10 vs best
           'high_quality_fast_inference_only_refit', #x10-100 vs best
           
           'good_quality_faster_inference_only_refit', #x4 vs prec
           
           'medium_quality_faster_train', #x10 prec
           'optimize_for_deployment',
    
    
    
           'good_quality_faster_inference_only_refit', 
           'optimize_for_deployment', 
           'medium_quality_faster_train',           
          ]

# hyperparameters = {
#     'NN': {}, 
#     'GBM': [
#          {'early_stopping_round' : 20},
        
#         {'extra_trees': True, 'AG_args': {'name_suffix': 'XT'}},
#             ], 
#     'CAT': {}, 
#     'RF': [
#         {'criterion': 'gini', 'AG_args': {'name_suffix': 'Gini', 'problem_types': ['binary', 'multiclass']}}, 
#         {'criterion': 'entropy', 'AG_args': {'name_suffix': 'Entr', 'problem_types': ['binary', 'multiclass']}}, 
#         {'criterion': 'mse', 'AG_args': {'name_suffix': 'MSE', 'problem_types': ['regression']}},
#             ], 
#     'XT': [
#         {'criterion': 'gini', 'AG_args': {'name_suffix': 'Gini', 'problem_types': ['binary', 'multiclass']}}, 
#         {'criterion': 'entropy', 'AG_args': {'name_suffix': 'Entr', 'problem_types': ['binary', 'multiclass']}}, 
#         {'criterion': 'mse', 'AG_args': {'name_suffix': 'MSE', 'problem_types': ['regression']}},
#             ], 
#     'KNN': [
#         {'weights': 'uniform', 'AG_args': {'name_suffix': 'Unif'}}, 
#         {'weights': 'distance', 'AG_args': {'name_suffix': 'Dist'}},
#             ], 
#     'custom': ['GBM' for elt in range(30)]
#     }

lgb_params =  {'early_stopping_round' : [20],
               'max_depth' : [-1,8,12],
             'n_estimators' : [50,100,200,500],
             'num_leaves': [6,15,25,45],
             'min_child_samples': [100,300,500], 
             'min_child_weight': [1e-5, 1e-2,  1,  1e2],
             'subsample': [0.2,0.4,0.6,0.8], 
             'colsample_bytree': [0.4,0.5,0.6],
             'reg_alpha': [0, 1e-1, 1, 2 ],
             'reg_lambda': [0, 1e-1, 1, 5]}

n_ensemble = 400
hyperparameters = {
    'GBM' : []
}

for i in range(n_ensemble):
    hyperparameters['GBM'].append(
        {key : np.random.choice(lgb_params[key]) for key in lgb_params}
    )
# print(hyperparameters)
    
time_limits = int(3600*3)
metric = 'roc_auc'
dir1 = 'agModels-predictClass'
predictor = task.fit(train_data=X, label='label', 
                     time_limits=time_limits,
                     tuning_data = X_val, 
                     eval_metric=metric, 
#                      presets='best_quality', 
                     output_directory=dir1,
                     auto_stack = False,   ## Create stacking
                     stack_ensemble_levels = 3,
                     hyperparameter_tune = False,  ## Tune the hyperparameters (longer)
                     hyperparameters = hyperparameters, # 'default', 'light', 'very_light', 'toy' ot dico
                     num_bagging_folds = 0,
                     search_strategy = 'skopt',
                     num_trials = 50,
#                      keep_only_best = False, ## Keep only one model
#                      refit_full = False,  ## Refit all models on the whole data
                    )

In [ ]:
predictor = task.load(dir1)
predictor.persist_models()

In [ ]:
predictor.leaderboard(silent = True)

In [ ]:
%%time
pred = predictor.predict(X_test.drop(columns = ['label']))
print(1)
proba = predictor.predict_proba(X_test.drop(columns = ['label']))

In [ ]:
import sklearn
print(sklearn.metrics.accuracy_score(y_test1, pred))
print(sklearn.metrics.roc_auc_score(y_test1, proba))
print(sklearn.metrics.roc_auc_score(y_test1, pred))

In [ ]:
# action = (pred>0.5)*1
action = pred
print(utility_metric(date_test,weights_test, y_test, pred))
print(utility_metric(date_test,weights_test, y_test, (y_test>0)*1))

In [ ]:
help(predictor.distill)

In [ ]:
student_models = predictor.distill(time_limits=180,
                                  hyperparameters = {'GBM': {}},
                                   augment_method = None,
                                   
                                  )  # specify much longer time-limits in real applications
print(student_models)
preds_student = predictor.predict(X_test.drop(columns = ['label']), model=student_models[0])
print(f"predictions from {student_models[0]}:", preds_student)
predictor.leaderboard(X_test, silent = True)

In [ ]:
model = './agModels-predictClass/models/LightGBMClassifier/model.pkl'
model = './agModels-predictClass/models/weighted_ensemble_k0_l1/model.pkl'
model = './agModels-predictClass/models/LightGBMRegressor_DSTL/model.pkl'

with open(model, 'rb') as f:
    distil = pickle.load(f)

In [ ]:
X_test.drop(columns = ['label']).columns

In [ ]:
help(distil.preprocess)

In [ ]:
import time
t0 = time.time()
for row in range(100):
#     a = predictor.predict(X_test.iloc[row:row+1])#, model=student_models[0])
    a = distil.predict(X_test.iloc[row:row+1])
print(time.time() - t0)

In [ ]:
%%time
pred = (distil.predict(X_test.drop(columns = ['label'])) >= 0.5)*1
proba = distil.predict_proba(X_test.drop(columns = ['label']))

In [ ]:
import sklearn
print(sklearn.metrics.accuracy_score(y_test1, pred))
print(sklearn.metrics.roc_auc_score(y_test1, proba))
print(sklearn.metrics.roc_auc_score(y_test1, pred))

In [ ]:
# action = (pred>0.5)*1
action = pred
utility_metric(date_test,weights_test, y_test, pred)

In [ ]:
action = pred
utility_metric(date_test,weights_test, y_test, (y_test>0)*1)

In [ ]:
lgb bag : full 2671, distilled 2504.6

In [ ]:
import autosklearn.classification
help(autosklearn.classification.AutoSklearnClassifier)

In [ ]:
!rm -r ./autosklearn_tmp
!rm -r ./autosklearn_output

In [ ]:
import autosklearn.classification
import autosklearn.metrics
import sklearn.model_selection
import sklearn.datasets
import sklearn.metrics
from datetime import datetime
now = datetime.now()
print("now =", now)
# X, y = sklearn.datasets.load_digits(return_X_y=True)
# X_train, X_test, y_train, y_test = \
#         sklearn.model_selection.train_test_split(X, y, random_state=1)

accuracy_scorer = autosklearn.metrics.make_scorer(
        name="accuracy",
        score_func=autosklearn.metrics.accuracy,
        optimum=1,
        greater_is_better=True,
        needs_proba=False,
        needs_threshold=False,
    )

auc_scorer = autosklearn.metrics.make_scorer(
        name="roc_auc",
        score_func=autosklearn.metrics.roc_auc,
        optimum=1,
        greater_is_better=True,
        needs_proba=True,
        needs_threshold=False,
    )


## Start
automl = autosklearn.classification.AutoSklearnClassifier(time_left_for_this_task=3600*5,
                                                          n_jobs = 2,
                                                          memory_limit = 12000,
                                                          tmp_folder  = './autosklearn_tmp',
                                                        output_folder = './autosklearn_output',
                                                          metric = auc_scorer,
                                                          scoring_functions  = [accuracy_scorer, auc_scorer],
                                                         )

automl.fit(X_train, y_train1, X_test = X_test, y_test = y_test1)

y_hat = automl.predict(X_test)

print(sklearn.metrics.accuracy_score(y_test1, y_hat))
print(sklearn.metrics.roc_auc_score(y_test1, y_hat))

In [ ]:
dir(au)

In [ ]:
y_hat = automl.predict(X_test)
y_proba = automl.predict_proba(X_test)[:,1]
print(sklearn.metrics.accuracy_score(y_test1, y_hat))
print(sklearn.metrics.roc_auc_score(y_test1, y_proba))

In [ ]:
# action = (pred>0.5)*1
# action = pred
utility_metric(date_test,weights_test, y_test, y_hat)

In [ ]:
y_hat

In [ ]:
%%time
for i in tqdm(range(10)):
    a = automl.predict(X_test[i:i+1])

In [ ]:
20000/14.6

In [ ]:
14.6/20000*2000000

In [ ]:
import lightgbm as lgb
clf = lgb.LGBMClassifier(max_depth = -1, n_estimators = 20000, n_jobs = 12, silent = False, early_stopping_rounds = 20)
clf.fit(X_train, (y_train>0)*1, eval_set =(X_test, (y_test>0)*1), eval_metric = 'auc', sample_weight=SAMPLE_WEIGHTS)

In [ ]:
# import lightgbm as lgb
# temp = 100
# clf = lgb.LGBMRegressor(max_depth = -1, n_estimators = 2000, n_jobs = 12, silent = False, early_stoping_rounds = 20)
# clf.fit(X_train, y_train*temp, eval_set =(X_test, y_test*temp), eval_metric = 'mse')


In [ ]:
import xgboost as xgb

clf = xgb.XGBClassifier(
    n_estimators=500,
    max_depth=11,
    learning_rate=0.05,
    subsample=0.9,
    colsample_bytree=0.7,
    random_state=2020,
    tree_method='gpu_hist',
    verbosity = 2
    
)

clf.fit(X_train.values, (y_train>0)*1, eval_metric="auc", eval_set=[(X_test.values,(y_test>0)*1)], early_stopping_rounds=10, sample_weight=SAMPLE_WEIGHTS)

In [ ]:
X_train.head()

In [ ]:
from catboost import CatBoostClassifier, CatBoostRegressor
clf = CatBoostClassifier(learning_rate=0.1, n_estimators=2000, early_stopping_rounds = 15, task_type="GPU", eval_metric = 'AUC')
clf.fit(X_train, (y_train>0)*1, eval_set =(X_test, (y_test>0)*1))

In [ ]:
X_train = X_train.values
X_test = X_test.values

kind = np.random.choice(range(X_train.shape[0]), size = 100000)
kind_test = np.random.choice(range(X_test.shape[0]), size = 5000)

X_train1 = X_train[kind]
y_train1 = y_train[kind]

X_test1 = X_test[kind_test]
y_test1 = y_test[kind_test]


X = np.concatenate([X_train1, X_test1])
y = np.concatenate([y_train1, y_test1])

train_idx = np.array(range(X_train1.shape[0]))
test_idx = np.array(range(X_test1.shape[0])) + train_idx.max()

split = [(train_idx, test_idx)]

sample_weights = abs((np.concatenate([y_train1, y_test1]) * np.concatenate([X_train1[:,0], X_test1[:,0]]))) + 1
sample_weights1 = abs(y_train1 * X_train1[:,0]) + 1
# def cv():
#     train_idx = np.array(range(X_train.shape[0]))
#     test_idx = np.array(range(X_test.shape[0])) + train_idx.max()
#     yield train_idx, test_idx

In [ ]:
# save((X, y, sample_weights, split), 'tpot_bench_data')
# (X, y, sample_weights, split)=load('tpot_bench_data')

In [ ]:
import numpy as np

# Check the TPOT documentation for information on the structure of config dicts
classifier_config_dict_light = {

    
    # Classifiers
#     'sklearn.naive_bayes.GaussianNB': {
#     },

#     'sklearn.naive_bayes.BernoulliNB': {
#         'alpha': [1e-3, 1e-2, 1e-1, 1., 10., 100.],
#         'fit_prior': [True, False]
#     },

#     'sklearn.naive_bayes.MultinomialNB': {
#         'alpha': [1e-3, 1e-2, 1e-1, 1., 10., 100.],
#         'fit_prior': [True, False]
#     },

#     'sklearn.tree.DecisionTreeClassifier': {
#         'criterion': ["gini", "entropy"],
#         'max_depth': range(1, 11),
#         'min_samples_split': range(2, 21),
#         'min_samples_leaf': range(1, 21)
#     },


#     'sklearn.neighbors.KNeighborsClassifier': {
#         'n_neighbors': range(1, 101),
#         'weights': ["uniform", "distance"],
#         'p': [1, 2]
#     },


#     'sklearn.linear_model.LogisticRegression': {
#         'penalty': ["l1", "l2"],
#         'C': [1e-4, 1e-3, 1e-2, 1e-1, 0.5, 1., 5., 10., 15., 20., 25.],
#         'dual': [True, False]
#     },

#     'sklearn.ensemble.ExtraTreesClassifier': {
#         'n_estimators': [100],
#         'criterion': ["gini", "entropy"],
#         'max_features': np.arange(0.05, 1.01, 0.05),
#         'min_samples_split': range(2, 21),
#         'min_samples_leaf': range(1, 21),
#         'bootstrap': [True, False]
#     },

#     'sklearn.ensemble.RandomForestClassifier': {
#         'n_estimators': [100],
#         'criterion': ["gini", "entropy"],
#         'max_features': np.arange(0.05, 1.01, 0.05),
#         'min_samples_split': range(2, 21),
#         'min_samples_leaf':  range(1, 21),
#         'bootstrap': [True, False]
#     },

#     'sklearn.ensemble.GradientBoostingClassifier': {
#         'n_estimators': [100],
#         'learning_rate': [1e-3, 1e-2, 1e-1, 0.5, 1.],
#         'max_depth': range(1, 11),
#         'min_samples_split': range(2, 21),
#         'min_samples_leaf': range(1, 21),
#         'subsample': np.arange(0.05, 1.01, 0.05),
#         'max_features': np.arange(0.05, 1.01, 0.05)
#     },
    
    'xgboost.XGBClassifier': {
        'n_estimators': [100,300,500],
        'max_depth': range(1, 11),
        'learning_rate': [1e-3, 1e-2, 1e-1, 0.5, 1.],
        'subsample': np.arange(0.05, 1.01, 0.05),
        'min_child_weight': range(1, 21),
        'n_jobs': [1],
        'verbosity': [0]
    },

#     'sklearn.linear_model.SGDClassifier': {
#         'loss': ['log', 'hinge', 'modified_huber', 'squared_hinge', 'perceptron'],
#         'penalty': ['elasticnet'],
#         'alpha': [0.0, 0.01, 0.001],
#         'learning_rate': ['invscaling', 'constant'],
#         'fit_intercept': [True, False],
#         'l1_ratio': [0.25, 0.0, 1.0, 0.75, 0.5],
#         'eta0': [0.1, 1.0, 0.01],
#         'power_t': [0.5, 0.0, 1.0, 0.1, 100.0, 10.0, 50.0]
#     },

#     'sklearn.neural_network.MLPClassifier': {
#         'alpha': [1e-4, 1e-3, 1e-2, 1e-1],
#         'learning_rate_init': [1e-3, 1e-2, 1e-1, 0.5, 1.]
#     },

    'lightgbm.LGBMClassifier':
        {
            'n_estimators': [100,300,500,700],
            'num_leaves' : [10,30,60],
            'min_child_samples' : [100,200,300,400,500],
            'min_child_weight': [1e-5, 1e-3, 1e-2, 1e-1, 1, 1e1, 1e2, 1e3, 1e4],
            'reg_alpha': [0, 1e-1, 1, 2, 5, 7, 10, 50, 100],
            'reg_lambda': [0, 1e-1, 1, 5, 10, 20, 50, 100],
            'early_stopping_rounds' : [None, 50,100,200]
    },
       
    # Preprocesssors
    'sklearn.preprocessing.Binarizer': {
        'threshold': np.arange(0.0, 1.01, 0.05)
    },

    'sklearn.cluster.FeatureAgglomeration': {
        'linkage': ['ward', 'complete', 'average'],
        'affinity': ['euclidean', 'l1', 'l2', 'manhattan', 'cosine']
    },

    'sklearn.preprocessing.MaxAbsScaler': {
    },

    'sklearn.preprocessing.MinMaxScaler': {
    },

    'sklearn.preprocessing.Normalizer': {
        'norm': ['l1', 'l2', 'max']
    },

    'sklearn.decomposition.PCA': {
        'svd_solver': ['randomized'],
        'iterated_power': range(1, 11)
    },

    'sklearn.kernel_approximation.RBFSampler': {
        'gamma': np.arange(0.0, 1.01, 0.05)
    },

    'sklearn.preprocessing.RobustScaler': {
    },

    'sklearn.preprocessing.StandardScaler': {
    },

    'tpot.builtins.ZeroCount': {
    },

    # Selectors
    'sklearn.feature_selection.SelectFwe': {
        'alpha': np.arange(0, 0.05, 0.001),
        'score_func': {
            'sklearn.feature_selection.f_classif': None
        }
    },

    'sklearn.feature_selection.SelectPercentile': {
        'percentile': range(1, 100),
        'score_func': {
            'sklearn.feature_selection.f_classif': None
        }
    },

    'sklearn.feature_selection.VarianceThreshold': {
        'threshold': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.2]
    }

}


In [ ]:
## Time bench gen 5 pop size 10 job 1 
## Test 3 000 

nb : 0.495 / 0.03  0.355
bernoullinb : 1.49 / 0.047  10
multinomialnb: fail
decisiontree: 13.7s / 0.003 7.58
knn: 27.4s / 1.05s 76
logreg: 6 / 0.016  25
extratree : 37 / 1.12
randforest : slow
gradientboost : 78s/1.78
xgbclass : 60 / 0.8
sgdclass :2.71 / 0.3
mlpclass : 38.8 / 0.7
lgbclass : 7.47 /0.1
    
( with nb)
binarizer:  0.852 / 57.3    7
feature_agglom: 0.750 / 0.014    25
maxabs: 0.35
minmax:  0.35
normalizer: 20.4
pca: 0.035
rbfsampler: 24.18
robustscaler: 0.35
stdscaler: 0.35
zerocount: 0.35
selectfwe: 0.35
selectpercentile: 3.75
variancetres : 0.35

In [ ]:
from tpot import TPOTClassifier
clf = TPOTClassifier(generations=15, population_size=50, 
                     offspring_size=None, mutation_rate=0.9, 
                     crossover_rate=0.1, scoring='roc_auc', cv=split, 
                     subsample=1.0, n_jobs=6, 
                     max_time_mins=200, max_eval_time_mins=2, 
                     random_state=None, 
                     config_dict=classifier_config_dict_light, 
                     template=None, warm_start=True, 
                     memory=None, use_dask=False, 
                     periodic_checkpoint_folder=None, 
                     early_stop=20, verbosity=2, 
                     disable_update_check=False, log_file=None)

In [ ]:
%%time
clf.fit(X, (y>0)*1, sample_weight=sample_weights)

In [ ]:
save(clf, 'tpot_end')

In [ ]:
clf.fitted_pipeline_

In [ ]:
%%time
end = X_test.shape[0]
pred = clf.predict(X_test[:end])

In [ ]:
pred.shape

In [ ]:
# action = (pred>0.5)*1
action = pred
utility_metric(date_test[:end],weights_test[:end], y_test[:end], action)

In [ ]:
clf.export('best pipeline.py')

In [ ]:
dir(clf)

In [ ]:
utility_metric(date_test[:end],weights_test[:end], y_test[:end], ( y_test[:end]>0)*1)

In [ ]:
import numpy as np
import pandas as pd
from lightgbm import LGBMClassifier
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline, make_union
from sklearn.preprocessing import RobustScaler
from tpot.builtins import StackingEstimator
from sklearn.preprocessing import FunctionTransformer
from copy import copy

exported_pipeline = make_pipeline(
    make_union(
        FunctionTransformer(copy),
        make_pipeline(
            PCA(iterated_power=7, svd_solver="randomized"),
            RobustScaler()
        )
    ),
    LGBMClassifier(early_stopping_rounds=None, min_child_samples=100, 
                   min_child_weight=0.001, n_estimators=100, 
                   num_leaves=10, reg_alpha=1, reg_lambda=50, 
                   silent = False, n_jobs = 12)
)

exported_pipeline.fit(X_train, (y_train>0)*1, lgbmclassifier__sample_weight=SAMPLE_WEIGHTS)

In [ ]:
from sklearn.ensemble import BaggingClassifier

clf = BaggingClassifier(base_estimator=make_pipeline(
                    make_union(
                        FunctionTransformer(copy),
                        make_pipeline(
                            PCA(iterated_power=7, svd_solver="randomized"),
                            RobustScaler()
                        )
                    ),
                    LGBMClassifier(early_stopping_rounds=None, min_child_samples=100, 
                                   min_child_weight=0.001, n_estimators=100, 
                                   num_leaves=10, reg_alpha=1, reg_lambda=50, 
                                   silent = False, n_jobs = 12)), 
            n_estimators=20, random_state=0)

clf.fit(X_train, (y_train>0)*1)#, sample_weight=SAMPLE_WEIGHTS)

In [ ]:
%%time
end = X_test.shape[0]
pred = automl.predict(X_test[:end])

action = pred
utility_metric(date_test[:end],weights_test[:end], y_test[:end], action)

In [ ]:
save(automl, 'autosklearn_4_h')

In [ ]:
plt.hist(pred)

In [ ]:
exported_pipeline

In [ ]:
import lightgbm as lgb
clf = lgb.LGBMClassifier(n_estimators=1000, n_jobs = 12, silent = False, 
                         min_child_samples=500, min_child_weight=10.0,
                         num_leaves=30, reg_alpha=0, reg_lambda=100,
                         early_stopping_rounds = 200)
clf.fit(X_train1, (y_train1>0)*1, eval_set =(X_test1, (y_test1>0)*1), eval_metric = 'auc', sample_weight=sample_weights1)

pred = clf.predict(X_test[:end])

action = pred
utility_metric(date_test[:end],weights_test[:end], y_test[:end], action)

## NN

In [ ]:
from tensorflow.keras.mixed_precision import experimental as mixed_precision
# policy = mixed_precision.Policy('mixed_float16')
policy = mixed_precision.Policy('float32')
mixed_precision.set_policy(policy)

In [3]:
(X_train, X_test, y_train, y_test, date_train, date_test, weights_train, weights_test) = load('splitted_dataset')
SAMPLE_WEIGHTS = abs((y_train * weights_train)) + 1
sample_weights = abs((np.concatenate([y_train, y_test]) * np.concatenate([weights_train, weights_test]))) + 1

In [4]:
X_train = X_train.values
# X_train['label'] = (y_train > 0)*1
y_train1 = (y_train > 0)*1

# X_val = X_train[date_train >= 420]
# X = X_train[date_train <420]

X_test = X_test.values#[:100]
y_test1 = (y_test > 0)*1

# X_test['label'] = y_test1

# X_test = X_test


In [5]:
## Masking and scaling
import tensorflow as tf
with tf.device('/CPU:0'):
    X_train = tf.keras.activations.sigmoid(X_train).numpy()
    X_test = tf.keras.activations.sigmoid(X_test).numpy()

# mask_train = np.random.choice([0,1], p = [0.15,0.85],size = (X_train.shape[0], X_train.shape[1]))
# mask_test = np.random.choice([0,1], p = [0.15,0.85],size = (X_test.shape[0], X_test.shape[1]))

# X_train, X_train_tar = X_train*mask_train, (1-mask_train)*X_train
# X_test, X_test_tar = X_test*mask_test, (1-mask_test)*X_test

In [6]:
from tabnet2 import *
import tensorflow as tf

size = X_train.shape[1]

# node = NODE(n_layers=1, units=1, depth=7, n_trees=80, link=tf.keras.activations.sigmoid)

tabnet_encoder = TabNet(num_features = size,  feature_dim = 256,   output_dim = 128,
        feature_columns = None,  n_step = 4,   n_total = 4,  n_shared = 2,
        relaxation_factor = 0.82,  bn_epsilon = 1e-5,  bn_momentum = 0.7,  bn_virtual_divider = 32
    )

tabnet_decoder = TabNet(num_features = 128,  feature_dim = 256,   output_dim = 128,
        feature_columns = None,  n_step = 4,   n_total = 4,  n_shared = 2,
        relaxation_factor = 0.82,  bn_epsilon = 1e-5,  bn_momentum = 0.7,  bn_virtual_divider = 32
    )

inputs_num = tf.keras.Input(shape = (size,), dtype = 'float32')#, batch_size = 1)

encoded, masks, loss_enc = tabnet_encoder(inputs_num, training = True)
decoded, masks_dec, loss_dec = tabnet_decoder(encoded, training = True)

pred = tf.keras.layers.Dense(size, activation = 'sigmoid')(decoded)
model = tf.keras.Model(inputs_num, pred)
model.add_loss(0.001*loss_enc)
model.add_loss(0.001*loss_dec)

In [7]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 131)]        0                                            
__________________________________________________________________________________________________
tab_net (TabNet)                ((None, 128), [(1, N 3542844     input_1[0][0]                    
__________________________________________________________________________________________________
tab_net_1 (TabNet)              ((None, 128), [(1, N 3537408     tab_net[0][0]                    
__________________________________________________________________________________________________
dense_32 (Dense)                (None, 131)          16899       tab_net_1[0][0]                  
_______________________________________________________________________________________

In [ ]:
loss_object = tf.keras.losses.CategoricalCrossentropy(
                        from_logits=False, label_smoothing=0, reduction=tf.keras.losses.Reduction.NONE,
                        name='categorical_crossentropy'
                    )

def masked_cat_loss(y_true, y_pred):
    mask = tf.cast(tf.math.logical_not(tf.math.equal(y_true, 0)),dtype = y_pred.dtype)
    y_true = tf.cast(y_true, y_pred.dtype)
    
    y_pred *= mask
    
#     print(y_true)
#     print(y_pred)
    loss = loss_object(y_true, y_pred)
    loss = tf.reduce_sum( - tf.math.log(1 - tf.math.minimum(tf.math.abs(y_true - y_pred), 0.99)), axis = -1)
#     print(loss)
    return tf.reduce_mean(loss)

def masked_mse_loss(y_true, y_pred):
    mask = tf.cast(tf.math.logical_not(tf.math.equal(y_true, 0)),dtype = y_pred.dtype)
    
    y_true = tf.cast(y_true, y_pred.dtype)
    
    loss = tf.reduce_sum(tf.pow((y_pred - y_true)*mask, 2), axis = -1)
    return tf.reduce_mean(loss)

In [ ]:
from tensorflow.keras.optimizers import Adam, SGD
model.compile(
        loss = 'categorical_crossentropy',
#         loss = masked_cat_loss, #'mse',
        optimizer = Adam(0.01),
#         metrics = ['accuracy', 'AUC'],
)

batch_size = 8192
epochs = 1000

ls = X_train.shape[0] // batch_size * batch_size
lt = X_test.shape[0] // batch_size * batch_size


from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ReduceLROnPlateau

datas = [date_test[:lt], weights_test[:lt], y_test[:lt]]


early = EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=8, verbose=1, 
                                                mode='min', restore_best_weights=True)

reduce = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1, 
                           mode='min', min_delta=0.0001, cooldown=0, min_lr=0)

# util = UtilityCallback(validation = (X_test[:lt].values, (y_test[:lt]>0)*1),batch_size = batch_size, datas = datas)
callbacks =[early, reduce]
# callbacks = []



model.fit(X_train[:ls],X_train[:ls], validation_data = (X_test[:lt], X_test[:lt]), 
          batch_size=batch_size, epochs=epochs, callbacks = callbacks,
         sample_weight = SAMPLE_WEIGHTS[:ls],
         )


Epoch 1/1000
208/208 [==============================] - 276s 1s/step - loss: 335.9805 - val_loss: 317.5090
Epoch 2/1000
208/208 [==============================] - 230s 1s/step - loss: 332.7187 - val_loss: 316.0152
Epoch 3/1000
208/208 [==============================] - 216s 1s/step - loss: 331.6372 - val_loss: 315.2913
Epoch 4/1000
208/208 [==============================] - 208s 1s/step - loss: 331.0882 - val_loss: 315.0257
Epoch 5/1000
208/208 [==============================] - 214s 1s/step - loss: 330.8979 - val_loss: 314.6793
Epoch 6/1000
208/208 [==============================] - 220s 1s/step - loss: 330.6208 - val_loss: 314.5102
Epoch 7/1000
208/208 [==============================] - 213s 1s/step - loss: 330.4488 - val_loss: 314.4377
Epoch 8/1000
208/208 [==============================] - 212s 1s/step - loss: 330.4380 - val_loss: 314.2663
Epoch 9/1000
208/208 [==============================] - 221s 1s/step - loss: 330.3317 - val_loss: 314.2026
Epoch 10/1000
208/208 [==============

In [ ]:
tabnet_encoder.save_weights('./tabencoder_unmasked/weights')

In [ ]:
from tabnet2 import *
import tensorflow as tf

size = X_train.shape[1]

# node = NODE(n_layers=1, units=1, depth=7, n_trees=80, link=tf.keras.activations.sigmoid)

tabnet_encoder = TabNet(num_features = size,  feature_dim = 256,   output_dim = 128,
        feature_columns = None,  n_step = 4,   n_total = 4,  n_shared = 2,
        relaxation_factor = 0.82,  bn_epsilon = 1e-5,  bn_momentum = 0.7,  bn_virtual_divider = 32
        )

tabnet_encoder.load_weights('./tabencoder/weights')

# for layer in tabnet_encoder.layers:
#     layer.trainable = False

inputs_num = tf.keras.Input(shape = (size,), dtype = 'float32')#, batch_size = 1)

encoded, masks, loss_enc = tabnet_encoder(inputs_num, training = False)

pred = tf.keras.layers.Dense(1, activation = 'sigmoid')(encoded)

model = tf.keras.Model(inputs_num, pred)

In [ ]:
model.load_weights('./tabnetcl/weights')

In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.optimizers import Adam, SGD
model.compile(
        loss = 'binary_crossentropy',
#         loss = masked_cat_loss, #'mse',
        optimizer = Adam(0.01),
        metrics = ['accuracy', 'AUC'],
)

batch_size = 8192
epochs = 1000

ls = X_train.shape[0] // batch_size * batch_size
lt = X_test.shape[0] // batch_size * batch_size


from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ReduceLROnPlateau

early = EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=8, verbose=1, 
                                                mode='min', restore_best_weights=True)

reduce = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1, 
                           mode='min', min_delta=0.0001, cooldown=0, min_lr=0)

# util = UtilityCallback(validation = (X_test[:lt].values, (y_test[:lt]>0)*1),batch_size = batch_size, datas = datas)
callbacks =[early, reduce]
# callbacks = []



model.fit(X_train[:ls],y_train1[:ls], validation_data = (X_test[:lt], y_test1[:lt]), 
          batch_size=batch_size, epochs=epochs, callbacks = callbacks,
         sample_weight = SAMPLE_WEIGHTS[:ls],
         )


In [ ]:
model.save_weights('./tabnetcl/weights')

In [ ]:
dir(tabnet_encoder)

In [ ]:
pred = model.predict(X_test, verbose = 1, batch_size = 8192)[:,0]

In [ ]:
plt.hist(pred, bins = 30)

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score
fpr, tpr, thresholds = roc_curve((y_test>0)*1, pred)
plt.plot(fpr, tpr, marker='.', label='Logistic')
gmeans = np.sqrt(tpr * (1-fpr))
ix = np.argmax(gmeans)
print('Best Threshold=%f, G-Mean=%.3f' % (thresholds[ix], gmeans[ix]))
roc_auc_score((y_test>0)*1, pred)

In [ ]:
action = (pred >= thresholds[ix])*1
utility_metric(date_test,weights_test, y_test, action)

## Distillation

In [ ]:
(X_train, X_test, y_train, y_test, date_train, date_test, weights_train, weights_test, train_feats, test_feats) =load('dataset_with_features')

In [ ]:
import tensorflow as tf
tf.config.optimizer.set_jit(True)
size = 132# X_train.shape[1]
inputs = tf.keras.Input(shape = (size,), dtype = 'float32')
training = False
x = tf.keras.layers.Dense(1024, activation = 'relu')(inputs)
x = tf.keras.layers.Dense(1024, activation = 'relu')(x)
# x = tf.keras.layers.BatchNormalization()(x, training = training)
x = tf.keras.layers.Dense(512, activation = 'relu')(x)
x = tf.keras.layers.Dense(512, activation = 'relu')(x)
# x = tf.keras.layers.BatchNormalization()(x, training = training)
x = tf.keras.layers.Dense(256, activation = 'relu')(x)
x = tf.keras.layers.Dense(256, activation = 'relu')(x)
x = tf.keras.layers.BatchNormalization()(x, training = training)
x = tf.keras.layers.Dense(128, activation = 'sigmoid')(x)

model = tf.keras.Model(inputs, x)

In [ ]:
model.load_weights('./emb_model/weights')

In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.optimizers import Adam, SGD
model.compile(
        loss = 'categorical_crossentropy',
#         loss = 'binary_crossentropy',
#         loss = 'mse',
        optimizer = Adam(0.01),
        metrics = ['accuracy', "mae"],
)

from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ReduceLROnPlateau

early = EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=8, verbose=1, 
                                                mode='min', restore_best_weights=True)

reduce = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1, 
                           mode='min', min_delta=0.0001, cooldown=0, min_lr=0)

callbacks = [early, reduce]
# callbacks = []

batch_size = 512
epochs = 1000

history = model.fit(X_train.values, train_feats, validation_data = (X_test.values, test_feats), epochs= epochs, batch_size=batch_size, callbacks = callbacks)
# history = model.fit(X_train.values, (y_train>0)*1, validation_data = (X_test.values, (y_test>0)*1), epochs= epochs, batch_size=batch_size, callbacks = callbacks)

In [ ]:
model.save_weights('./emb_model/weights')

In [ ]:
tf = model.predict(X_train.values, verbose = 1, batch_size = 512)
vf = model.predict(X_test.values, verbose = 1, batch_size = 512)

In [ ]:
save((tf, vf), 'nn_embedding')

In [ ]:
(tf, vf) = load('nn_embedding')

In [ ]:
plt.hist(vf[21], bins = 20)

In [ ]:
from catboost import CatBoostClassifier, CatBoostRegressor
clf = CatBoostClassifier(learning_rate=0.1, n_estimators=20000, early_stopping_rounds = 15, task_type="GPU", eval_metric = 'AUC')
clf.fit(tf, (y_train>0)*1, eval_set =(vf, (y_test>0)*1))

In [ ]:
import lightgbm as lgb
clf = lgb.LGBMClassifier(max_depth = -1, n_estimators = 30000, n_jobs = 12, silent = False, early_stopping_rounds = 20)
clf.fit(tf, (y_train>0)*1, eval_set =(vf, (y_test>0)*1), eval_metric = 'auc')

In [ ]:
save(clf, 'lightgbm_sup')

In [ ]:
clf = load('lightgbm_sup')

In [ ]:
X_test.head()

In [ ]:
@tf.function(experimental_compile=True)
def model_fn(x):
    return model(x, training = False)

# def create_and_run_graph():
#     x = tf.placeholder(tf.float32, name='x')
#     result = tf.xla.experimental.compile(computation=model_fn, inputs=(x))[0]
#     return result

In [ ]:
model_fn(X_test[0:1][cols].values.astype('float32'))

In [ ]:

cols = ['date', 'weight'] + ['feature_'+str(i) for i in range(130)]
actions = []
lt = 20000
for i in tqdm(range(lt)):
    x = X_test[i:i+1]
    x = x[cols].values.astype('float32')
    x[np.isnan(x)] = -100
    with tf.device('/GPU:0'):
        e = model(x, training = False)
    p = clf.predict(e)
    actions.append(p)

In [ ]:
action = np.array(actions)[:,0]

In [ ]:
e

In [ ]:
x1 = x[cols].values

In [ ]:
x1[np.isnan(x1)] = -100

In [ ]:
# action = (pred>0.505679)*1
# action = clf.predict(vf)
# action = (y_test[:lt]>0)*1
# lt = test_feats.shape[0]
utility_metric(date_test[:lt],weights_test[:lt], y_test[:lt], action)

In [ ]:
x = X_test.values.astype('float32')

with tf.device('/GPU:0'):
    for i in tqdm(range(500)):
        p = model(x[i:i+1, :size], training = False)

In [ ]:
# model.load_weights('./checkpoints/tabnet1/tabnet_val_auc_0.8131')
model.load_weights('./checkpoints/int/weights')

In [ ]:
from sklearn.metrics import roc_curve
fpr, tpr, thresholds = roc_curve((y_test[:lt]>0)*1, pred)
plt.plot(fpr, tpr, marker='.', label='Logistic')
gmeans = np.sqrt(tpr * (1-fpr))
ix = np.argmax(gmeans)
print('Best Threshold=%f, G-Mean=%.3f' % (thresholds[ix], gmeans[ix]))
roc_auc_score((y_test[:lt]>0)*1, pred)

In [ ]:
(y_test[:lt]>0)*1

In [ ]:
# action = (pred>0.505679)*1
action = clf.predict(test_feats)
# action = (y_test[:lt]>0)*1
lt = test_feats.shape[0]
utility_metric(date_test[:lt],weights_test[:lt], y_test[:lt], action)

In [ ]:
plt.hist(pred, bins = 50)

In [ ]:
import optuna
def objective(trial):
    x = trial.suggest_uniform('x', 0, 1)
    action = (pred>x)*1
    utility = accuracy_score((y_test[:lt]>0), (pred >x)*1)     #utility_metric(date_test[:lt],weights_test[:lt], y_test[:lt], action)
    print(utility)
    return -utility

In [ ]:
import optuna
def objective(trial):
    x = trial.suggest_uniform('x', 0, 1)
    action = action = (pred>x)*1
    utility = utility_metric(date_test[:lt],weights_test[:lt], y_test[:lt], action)
    print(utility)
    return -utility

In [ ]:
study = optuna.create_study()
study.optimize(objective, n_trials=100)

In [ ]:
tres = study.best_params['x']
print(tres)
action = (pred>tres)*1
utility_metric(date_test[:lt],weights_test[:lt], y_test[:lt], action)

## Benchmark
max possible : 45093.72
lgb  mse est : 2000 t = 100 tres = -0.004808  utility : 13721  mse : 4.976 not stopped

lgb bce est : 2000 auc : 0.6912     utility : 17016   tres : 0.5005077   tres_utility : 17115 not stopped

catboost bce num est : 2000 auc : 0.6465   utility : 13191.73  tres : 0.5029   tres_utility : 13242 not stopped

catboost bce cat est : 2000 auc : 0.6814  utility : 17045  not stopped

catboost bce cat est : 7500 auc : 0.7125  utility : 20257 stopped

catboost mse num est : 2000 RMSE : 26.75  utility : 2653

xgboost bce num est : 500  utility : 19857

tabnet bce num nstep : 3, fd 256 od 128 , AUC : 0.81, utility : 26964, tres : 0.508

tabnet bce num nstep : 4, fd 256 od 128 , AUC : 0.79, utility : 24369, tres :

tabnet fast nstep : 2, fd 256 of 128, AUC : 0.75